In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
nltk.download('stopwords')


c:\Users\User\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [6]:
%pip install pymystem3


������ ������� ��࠭��: 1251
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from pymystem3 import Mystem
mystem = Mystem()
def lemmatize(token: str) -> str:
    return ''.join(mystem.lemmatize(token)).strip()


In [ ]:
import pandas as pd
df = pd.read_csv('qa.csv', sep=';')
texts = df['text'].astype(str).tolist()

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

russian_stop = set(stopwords.words("russian"))
stemmer = SnowballStemmer("russian")


greetings_re = re.compile(r'\b(?:добрый день|здравствуйте|спасибо|мтс|коллеги|пожалуйста|подскажите|просьба|необходимо|консультация|прошу|рублей|руб|умный бизнес)\b', flags=re.IGNORECASE)

def preprocess_fast(text: str) -> str:
    text = greetings_re.sub("", text.lower())
    tokens = re.findall(r"[а-яё]+", text.lower())
    stems = [
        stemmer.stem(token)
        for token in tokens
        if token not in russian_stop and len(token) > 2
    ]
    return " ".join(stems)

processed_texts = [preprocess_fast(t) for t in texts]
print(f"Документов после очистки и стемминга: {len(processed_texts)}")


Документов после очистки и стемминга: 1498


In [ ]:

from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

In [66]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("cointegrated/rubert-tiny2")


In [ ]:
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=15, min_samples=1)
from umap import UMAP

umap_model = UMAP(n_neighbors=10, n_components=5, min_dist=0.1, metric="cosine")

In [ ]:
from bertopic import BERTopic

topic_model = BERTopic(
    embedding_model=embedding_model, 
    language="russian",
    nr_topics="auto",
    umap_model=umap_model,
    hdbscan_model=hdbscan_model
)

In [75]:
topics, probs = topic_model.fit_transform(processed_texts)

In [77]:
pd.set_option('display.max_colwidth', None)
topic_info = topic_model.get_topic_info()
topic_info_filtered = topic_info[['Topic', 'Count', 'Name', 'Representation']]
display(topic_info_filtered)

,Topic,Count,Name,Representation
0,-1,382,-1_услуг_тариф_номер_скидк,"[услуг, тариф, номер, скидк, пакет, дан, абонент, подключен, котор, возможн]"
1,0,184,0_интернет_опц_трафик_услуг,"[интернет, опц, трафик, услуг, скорост, квот, подключен, границ, пакет, клиент]"
2,1,171,1_услуг_дан_минут_котор,"[услуг, дан, минут, котор, месяц, возможн, форис, номер, счет, эт]"
3,2,162,2_номер_услуг_смс_абонент,"[номер, услуг, смс, абонент, клиент, подключен, дан, запрос, групп, вызов]"
4,3,131,3_пакет_плат_списан_услуг,"[пакет, плат, списан, услуг, месяц, период, тариф, минут, скидк, баланс]"
5,4,130,4_услуг_продаж_приказ_нужн,"[услуг, продаж, приказ, нужн, как, клиент, отчет, котор, контракт, категор]"
6,5,109,5_тариф_тарифн_план_нов,"[тариф, тарифн, план, нов, цен, опц, стран, корп, приказ, регион]"
7,6,40,6_блокировк_проверк_принудительн_блокировок,"[блокировк, проверк, принудительн, блокировок, частичн, добровольн, номер, пресейл, налич, как]"
8,7,32,7_номер_телефон_городск_смс,"[номер, телефон, городск, смс, звонк, регион, вызов, абонент, домашн, пакет]"
9,8,28,8_доставк_роуминг_карт_международн,"[доставк, роуминг, карт, международн, интернет, корп, услуг, опц, легк, удален]"


In [78]:
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
# %% Построение и обучение BERTopic
# texts — ваш список строк (можно пустые, необработанные, BERTopic сам приведёт к нужному виду)
topic_model = BERTopic(
    embedding_model=embedding_model,
    language="russian",       # учтёт стоп-слова и морфологию
    #calculate_probabilities=True,
    nr_topics="auto"          # авто‑подбор числа тем
)

topics, probs = topic_model.fit_transform(processed_texts)

In [79]:
pd.set_option('display.max_colwidth', None)
topic_info = topic_model.get_topic_info()
topic_info_filtered = topic_info[['Topic', 'Count', 'Name', 'Representation']]
display(topic_info_filtered)

,Topic,Count,Name,Representation
0,-1,648,-1_услуг_номер_пакет_абонент,"[услуг, номер, пакет, абонент, клиент, возможн, котор, подключен, дан, тариф]"
1,0,187,0_услуг_номер_абонент_смс,"[услуг, номер, абонент, смс, нужн, котор, продаж, как, вызов, телефон]"
2,1,140,1_интернет_трафик_опц_квот,"[интернет, трафик, опц, квот, услуг, границ, безлимитн, пакет, бизнес, бит]"
3,2,108,2_плат_скидк_ежемесячн_тариф,"[плат, скидк, ежемесячн, тариф, месяц, списан, период, услуг, подключен, клиент]"
4,3,104,3_тариф_тарифн_план_нов,"[тариф, тарифн, план, нов, цен, стран, опц, регион, возможн, номер]"
5,4,79,4_услуг_дан_защитник_запрет,"[услуг, дан, защитник, запрет, абонент, номер, регион, вызов, информирован, москв]"
6,5,65,5_источник_продукт_счетчик_настройк,"[источник, продукт, счетчик, настройк, дан, заявк, каталог, как, форис, котор]"
7,6,37,6_блокировк_проверк_принудительн_блокировок,"[блокировк, проверк, принудительн, блокировок, частичн, добровольн, номер, пресейл, как, настройк]"
8,7,28,7_участник_групп_инициатор_акцептор,"[участник, групп, инициатор, акцептор, донор, забугорищ, пакет, сем, интернет, популярн]"
9,8,25,8_состоян_бо_мид_главупдк,"[состоян, бо, мид, главупдк, как, ве, изменен, лайт, наход, отгружа]"
